In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')
# データフレームを表示して確認
df.head()

,カテゴリー,商品コード,商品名,売上日,単価,数量,原価
0,食品,1001,りんご,2023-01-01,200,50,120
1,食品,1002,バナナ,2023-01-01,150,100,80
2,食品,1003,牛乳,2023-01-02,180,80,100
3,衣服,2001,Tシャツ,2023-01-02,1500,20,800
4,衣服,2002,ジーンズ,2023-01-03,5000,10,2500


In [3]:
# 2. データをLLM用にテキスト形式に変換
# データフレーム全体を文字列に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"
# 表示して確認
print(prompt_text)

売上データ:
    カテゴリー 商品コード      商品名         売上日    単価   数量    原価
0      食品  1001      りんご  2023-01-01   200   50   120
1      食品  1002      バナナ  2023-01-01   150  100    80
2      食品  1003       牛乳  2023-01-02   180   80   100
3      衣服  2001     Tシャツ  2023-01-02  1500   20   800
4      衣服  2002     ジーンズ  2023-01-03  5000   10  2500
..    ...   ...      ...         ...   ...  ...   ...
235    衣服  2077   レインパンツ  2023-04-28  2000   18  1000
236    食品  1085      ザクロ  2023-04-29   600   40   300
237   日用品  3077    バスブラシ  2023-04-29   400   60   200
238    衣服  2078  レインシューズ  2023-04-30  2500   15  1250
239    食品  1086    ココナッツ  2023-04-30   300   80   150

[240 rows x 7 columns]
この売上データの傾向を分析してください。


In [4]:
# 3. OpenAI APIの呼び出し

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

この売上データを分析するにあたって、以下の観点からのインサイトを提供します。

### 1. カテゴリー別売上分析
- **食品、衣服、日用品**の3つの主要なカテゴリーが確認できます。
- **売上総額**: 各カテゴリーごとの売上総額を計算し、最も売上が高いカテゴリーを特定します。

### 2. 商品別パフォーマンス
- 各商品ごとの売上、数量、利益（利益 = 売上 - 原価）を評価します。
- ベストセラー商品（売上や数量が最も多い商品）を特定します。

### 3. 月別売上トレンド
- 売上日を基に月別の売上トレンドを作成し、特定の月での売上の急増や減少を確認します。
- 特定の季節やイベント（例: 年末、バレンタインデーなど）による影響を探ります。

### 4. 利益率の分析
- 各商品やカテゴリーの**利益率**（利益率 = (売上 - 原価) / 売上）を計算し、利益率が高い商品を特定します。
- 利益率が高いが売上が低い商品に対するマーケティング戦略を考えます。

### 5. 単価と数量の相関分析
- 単価が高い商品が数量的にどう販売されているかを分析します。高単価商品は短期間での売上が高いのか、逆に低単価商品が売れやすいのかを見ることができます。

### 分析結果の例
例えば、以下のような分析結果が得られるかもしれません。
- **食品カテゴリー**が最も売上が高く、特にりんごとバナナの人気が高い。
- **衣服カテゴリー**の中では、Tシャツの売上が突出して高い。
- 4月は全体的に売上が増加している傾向が見られ、特に清掃関連商品（バスブラシなど）が好調。
- 一部の商品の利益率が顕著に高く、これらの商品は更なるマーケティング支援が求められる。

### 提案
1. **プロモーション戦略**: ベストセラー商品や高利益率の商品に対するターゲット広告やキャンペーンの強化を図ります。
2. **在庫調整**: 売上の波に応じた在庫調整を行い、欠品や余剰在庫を防ぎます。
3. **新商品開発**: トレンドや人気商品を踏まえた新商品の開発を検討します。

このような分析を通じて、企業の成長を支援するための戦略を立案することが可能です。データに基づいて具体的なアクションプランを提案することが大切です。


In [5]:
# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])
print(df_out)

                                                   結果
0              この売上データを分析するにあたって、以下の観点からのインサイトを提供します。
1                                                    
2                                   ### 1. カテゴリー別売上分析
3                 - **食品、衣服、日用品**の3つの主要なカテゴリーが確認できます。
4   - **売上総額**: 各カテゴリーごとの売上総額を計算し、最も売上が高いカテゴリーを特定します。
5                                                    
6                                   ### 2. 商品別パフォーマンス
7               - 各商品ごとの売上、数量、利益（利益 = 売上 - 原価）を評価します。
8                     - ベストセラー商品（売上や数量が最も多い商品）を特定します。
9                                                    
10                                    ### 3. 月別売上トレンド
11        - 売上日を基に月別の売上トレンドを作成し、特定の月での売上の急増や減少を確認します。
12          - 特定の季節やイベント（例: 年末、バレンタインデーなど）による影響を探ります。
13                                                   
14                                      ### 4. 利益率の分析
15  - 各商品やカテゴリーの**利益率**（利益率 = (売上 - 原価) / 売上）を計算し、...
16                - 利益率が高いが売上が低い商品に対するマーケティング戦略を考えます。
17                          

In [6]:
# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

In [7]:
# ワークフロー化
print("処理を開始します。")

# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')

# 2. データをLLM用にテキスト形式に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"

# 3. OpenAI APIの呼び出し
# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"
# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])

# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

print("Excelファイルに分析結果を保存しました。")

処理を開始します。
Excelファイルに分析結果を保存しました。
